<a href="https://colab.research.google.com/github/mfilipak/AFRAC_IA/blob/main/Doa%C3%A7%C3%A3o_XML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import shutil
!git clone https://github.com/mfilipak/AFRAC_IA.git
shutil.move("AFRAC_IA","../MYTMP")
!cp -r ../MYTMP/** .
!ls

**Importar os Pacotes Necessários**

---



In [ ]:
import os
import glob

In [ ]:
try:
  from flask import Flask, render_template, request
except:
  !pip install flask
  from flask import Flask, render_template, request

In [ ]:
try:
  from flask_dropzone import Dropzone
except:
  !pip install flask_dropzone
  from flask_dropzone import Dropzone

In [ ]:
try:
  from flask_ngrok import run_with_ngrok
except:
  !pip install flask_ngrok
  from flask_ngrok import run_with_ngrok

**Criar as pastas temporárias**

---



In [ ]:
if not os.path.exists('uploads'):
  os.makedirs('uploads')  

**Importar pacotes do Google e Criar os objetos de Conexão**

---



In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

gauth = GoogleAuth()           
drive = GoogleDrive(gauth)  

**Instanciar Servidor para receber os arquivos**

---

Após este passo, antes de seguir, interromper a excecução no menu Ambiente de Excecução.

In [ ]:
basedir = os.path.abspath(os.path.dirname("__file__"))

app = Flask(__name__)
app.config.update(
    DROPZONE_ADD_REMOVE_LINKS = True,
    DROPZONE_ALLOWED_FILE_CUSTOM = True,
    DROPZONE_ALLOWED_FILE_TYPE = '.xml, .zip',
    DROPZONE_DEFAULT_MESSAGE = "Solte arquivos \"XML ou ZIP\" aqui para enviar<br/><br/>ou<br/><br/><button type=\'button\'>Selecionar arquivos</button><br/><br>Tamanho máximo de upload de arquivo: 1 MB",
    UPLOADED_PATH = os.path.join(basedir,'uploads'), 
    DROPZONE_MAX_FILES_SIZE = 1024,
    DROPZONE_TIMEOUT = 5 * 60 * 1080)

dropzone = Dropzone(app)
run_with_ngrok(app)  


@app.route('/', methods=['POST','GET'])
def upload():
    if request.method == 'POST':
        f = request.files.get('file')
        f.save(os.path.join(app.config['UPLOADED_PATH'],f.filename))
    return render_template('index.html')


app.run()


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://2b786190a5d6.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [04/May/2021 19:32:28] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [04/May/2021 19:32:29] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [04/May/2021 19:33:03] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [04/May/2021 19:33:59] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [04/May/2021 19:33:59] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [04/May/2021 19:34:02] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [04/May/2021 19:36:11] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [04/May/2021 19:36:51] "POST /shutdown HTTP/1.1" 404 -
127.0.0.1 - - [04/May/2021 19:36:55] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [04/May/2021 19:37:20] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [04/May/2021 19:38:20] "POST /shutdown HTTP/1.1" 404 -
127.0.0.1 - - [04/May/2021 19:38:23] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [04/May/2021 19:38:46] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [04/May/2021 19:42:01] "POST / HTTP/1.1" 200 -


**Configurar Variavel de Ambiente com Credencial do Google Cloud.**

---

O Arquivo Json de Credenciais deve ser enviado para o Colab ante de executar o próximo passo.

In [ ]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/content/doacaoxml-2c6cbb88806a.json' 
!echo $GOOGLE_APPLICATION_CREDENTIALS

/content/doacaoxml-2c6cbb88806a.json


**Criar o Bucket ou apenas instanciar objeto, caso o bucket já tenha sido criado**

---



In [ ]:
from google.cloud import storage

def create_bucket_class_location(bucket_name):
    """Create a new bucket in specific location with storage class"""
    # bucket_name = "your-new-bucket-name"

    storage_client = storage.Client()

    bucket = storage_client.bucket(bucket_name)
    bucket.storage_class = "STANDARD"
    new_bucket = storage_client.create_bucket(bucket)

    print(
        "Created bucket {} in {} with storage class {}".format(
            new_bucket.name, new_bucket.location, new_bucket.storage_class
        )
    )
    return new_bucket

try:
  bucket = create_bucket_class_location('doacao_xml')
except:
  storage_client = storage.Client()
  bucket = storage_client.get_bucket('doacao_xml')

**Enviar os arquivos doados para o Bucket no Google Cloud**

---



In [ ]:
os.chdir('/content/uploads')
upload_file_list = []

res = [f for f in glob.glob("*.*") if ".xml" in f or ".zip" in f ]
for f in res:
  blob = bucket.blob('/content/uploads/' + f) 
  blob.upload_from_filename('/content/uploads/' +  f)
  print(f"Armazenando arquivo {f} no Google Cloud")


Armazenando arquivo xmlDfe-33200531698759000202650070000271731130573000 - Copia.xml no Google Cloud
Armazenando arquivo xmlDfe-33210305255141000179650110000539531589179911.xml no Google Cloud
Armazenando arquivo xmlDfe-33200531698759000202650070000271731130573000.xml no Google Cloud
